# **A study on Regression Algorithm applied to Insurance dataset**
**This notebook is a form to practice my knowledge in data science**

The notebook walks us through a workflow for solving a problem with a vehicle insurance regression algorithm.

The main purpose of this notebook is to serve as a step-by-step workflow guide, allowing me to review this notebook myself and serve as a study for future cases.

## Workflow stages
The solution workflow goes through four stages.
1.   Load the Data.
2.   Data pre-processing.
3.   Creating the regression model.
4.   Model predictions.

##1) Load the Data

In [0]:
#Import the library that creates the spark section
from pyspark.sql import SparkSession 

In [0]:
#Starts the section for using spark
spark = SparkSession.builder.appName("LinearRegression").getOrCreate()

In [0]:
%fs ls /FileStore/tables

path,name,size,modificationTime
dbfs:/FileStore/tables/movies-1.csv,movies-1.csv,494431,1662647401000
dbfs:/FileStore/tables/movies-2.csv,movies-2.csv,494431,1662647447000
dbfs:/FileStore/tables/movies.csv,movies.csv,494431,1662647363000
dbfs:/FileStore/tables/ratings.csv,ratings.csv,2483723,1662647649000
dbfs:/FileStore/tables/u.data,u.data,1979173,1662474869000


In [0]:
#Get the directory containing the file to use
dir="dbfs:/FileStore/shared_uploads/matheusmmmp@hotmail.com/regressaoLinear.csv"

In [0]:
#Defines the schema
from pyspark.sql.types import *  #Import of types
schema=StructType().add("X",IntegerType(),True).add("Y",StringType(),True)  


In [0]:
#Read CSV files with defined schema
insuranceClaim = spark.read.format('csv').schema(schema).options(header='true',delimiter=';').load(dir)

In [0]:
insuranceClaim.printSchema()

root
 |-- X: integer (nullable = true)
 |-- Y: string (nullable = true)



In [0]:
insuranceClaim.show(5)

+---+-----+
|  X|    Y|
+---+-----+
|108|392,5|
| 19| 46,2|
| 13| 15,7|
|124|422,2|
| 40|119,4|
+---+-----+
only showing top 5 rows



##2) Data pre-processing

In [0]:
import pyspark.sql.functions as F
insuranceClaim=insuranceClaim.select(F.col('X').alias("Policy"), F.col('Y').alias("Price"))  #Rename header names

In [0]:
insuranceClaim.show(5)

+------+-----+
|Policy|Price|
+------+-----+
|   108|392,5|
|    19| 46,2|
|    13| 15,7|
|   124|422,2|
|    40|119,4|
+------+-----+
only showing top 5 rows



In [0]:
#Formatted the value from "," to "."
insuranceClaimFormatted=insuranceClaim.withColumn("Price_Formatted", F.regexp_replace(F.col("Price"), "[,]", "."))  

In [0]:
insuranceClaimFormatted.show(5)

+------+-----+---------------+
|Policy|Price|Price_Formatted|
+------+-----+---------------+
|   108|392,5|          392.5|
|    19| 46,2|           46.2|
|    13| 15,7|           15.7|
|   124|422,2|          422.2|
|    40|119,4|          119.4|
+------+-----+---------------+
only showing top 5 rows



In [0]:
#Formatted type string to numeric
insuranceClaimFinal=insuranceClaimFormatted.select(F.col('Policy'),F.col('Price_Formatted'), insuranceClaimFormatted.Price_Formatted.cast('float').alias('Price_Float'))

In [0]:
insuranceClaimFinal.show(5)

+------+---------------+-----------+
|Policy|Price_Formatted|Price_Float|
+------+---------------+-----------+
|   108|          392.5|      392.5|
|    19|           46.2|       46.2|
|    13|           15.7|       15.7|
|   124|          422.2|      422.2|
|    40|          119.4|      119.4|
+------+---------------+-----------+
only showing top 5 rows



In [0]:
insuranceClaimFinal.printSchema()

root
 |-- Policy: integer (nullable = true)
 |-- Price_Formatted: string (nullable = true)
 |-- Price_Float: float (nullable = true)



In [0]:
insuranceClaimFinal.describe().show()

+-------+------------------+-----------------+-----------------+
|summary|            Policy|  Price_Formatted|      Price_Float|
+-------+------------------+-----------------+-----------------+
|  count|                63|               63|               63|
|   mean|22.904761904761905|98.18730158730159|98.18730196877131|
| stddev| 23.35194561605733|87.32755263404981|87.32755386147541|
|    min|                 0|                0|              0.0|
|    max|               124|             98.1|            422.2|
+-------+------------------+-----------------+-----------------+



After loading the dataset, the analysis starts.
For Linear Regression, you need to import the Vector Assembler from Pyspark's MLlib module.

Vector Assembler is a transformer that assembles all resources into a vector from multiple columns that contain double type.

In [0]:
#Convert data (rows) into vectors
from pyspark.ml.feature import VectorAssembler  #Importing the library responsible for creating the vector from the column

assembler = VectorAssembler(inputCols=['Policy'], outputCol='features')  #Defines the object for conversion
df_insurance = assembler.transform(insuranceClaimFinal) #Apply the conversion
df_insurance.printSchema()

root
 |-- Policy: integer (nullable = true)
 |-- Price_Formatted: string (nullable = true)
 |-- Price_Float: float (nullable = true)
 |-- features: vector (nullable = true)



In [0]:
df_insurance = df_insurance.select(['features','Price_Float'])
df_insurance.show(5)

+--------+-----------+
|features|Price_Float|
+--------+-----------+
| [108.0]|      392.5|
|  [19.0]|       46.2|
|  [13.0]|       15.7|
| [124.0]|      422.2|
|  [40.0]|      119.4|
+--------+-----------+
only showing top 5 rows



##3) Creating the regression model
For the Linear Regression algorithm, the LinearRegression will be used from Pyspark's MLlib module, where a model is defined and trained

In [0]:
 #Library that contains the regression model
from pyspark.ml.regression import LinearRegression 
lr = LinearRegression(maxIter=10, labelCol='Price_Float') #Defines the object to be used for regression
lrModel = lr.fit(df_insurance)

In [0]:
#Angular and linear coefficients (a and b) of the regression line
print(f'Intercept: {lrModel.intercept}\nAngular Coefficient: {lrModel.coefficients.values}')

Intercept: 19.994485035718697
Angular Coefficient: [3.41382361]


Analysis of the statistical summary of the model
 
*   Explained Varience: Represents the sum of squares of explained deviation values.
*   R2: This is the coefficient of determination in which, through the data presented, it is possible to identify the linear model explains 83,
33% of the variance of the dependent variable from the independent variables included in the linear model.
*   Mean Squared Error: It is the measure of how far the sample points deviate from the regression line, it is commonly used to verify the accuracy of models

In [0]:
#Model statistics print
modelsummary = lrModel.summary

print("Explained Varience:", modelsummary.explainedVariance)
print('R_2: ', modelsummary.r2)
print('Mean Squared Error: ',modelsummary.meanSquaredError)

modelsummary.residuals.show(5)


Explained Varience: 6254.310467068246
R_2:  0.8333466721037774
Mean Squared Error:  1250.7419636077982
/databricks/spark/python/pyspark/sql/context.py:134: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
+-------------------+
|          residuals|
+-------------------+
| 3.8125652674761454|
| -38.65713283055052|
| -48.67419213440233|
|-21.108600258352624|
| -37.14742784198987|
+-------------------+
only showing top 5 rows



##4) Model prediction
Through the dataset it is possible to compare the values ​​already informed with the predictions of the model

In [0]:
modelsummary.predictions.show(5)

+--------+------------------+------------------+
|features|       Price_Float|        prediction|
+--------+------------------+------------------+
| [108.0]|             392.5|388.68743473252385|
|  [19.0]| 46.20000076293945| 84.85713359348998|
|  [13.0]|15.699999809265137| 64.37419194366747|
| [124.0]|422.20001220703125| 443.3086124653839|
|  [40.0]| 119.4000015258789|156.54742936786877|
+--------+------------------+------------------+
only showing top 5 rows

